# Testing the model
We will be using the five images that Flickr27 has set aside for validation. For training and validating the model, we had images in seperate class folder. For testing the model, all the test images have been moved into one single folder. 

I have also added the names of the classes to each of the images so that we can compare the files names with the prediction. To validate the results, we would not have to open the image files manually. Also, there is no code that uses the name of the file as a variable for prediction

In [1]:
import numpy as np
import cv2 as cv
import os
import time
import pandas as pd

import skimage.io
import skimage.transform

from keras.models import model_from_json
from keras.models import load_model
from keras import optimizers
from keras.preprocessing.image import img_to_array
from keras.callbacks import ModelCheckpoint

import imutils

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [1]:
#Moved the model and weights file into a structure.
modelFileName = 'Execution History//20180814//NewIV3-150x150x1-3C.h5'
weightFileName = 'Execution History//20180814//NewIV3-150x150x1-3C-Run1.hdf5'

#This is the folder in which the test images are present
testImageFolder = 'Flickr27//TestImages'

#These are the classes I am testing for. The assumption is that one of these logos will exist in the image.
class_names = ['Apple', 'Fedex', 'Google'];

#A simple counter system to see how many files were present and how many were predicted correctly.
acn0 = 0;
acn1 = 0;
acn2 = 0;
acn3 = 0;
acn4 = 0;
tcn0 = 0;
tcn1 = 0;
tcn2 = 0;
tcn3 = 0;
tcn4 = 0;

In [2]:
def loadModel(modelFileName, weightFileName):
    #Load the model from the H5 file.
    model = load_model(modelFileName)
    
    #Load the weights from the HDF5 file.
    model.load_weights(weightFileName)
    print("Loaded model from disk")
    
    return model

def predictGrey (image,tcn0,acn0,tcn1,acn1,tcn2,acn2,tcn3,acn3,tcn4,acn4):

    #Image has been found, increase the class counter by 1.
    if class_names[0] in image:
        acn0+=1;
    elif class_names[1] in image:
        acn1+=1;
    elif class_names[2] in image:
        acn2+=1;
        
    #Reading the image            
    size = 150
    img = cv.imread(image)
    
    #Converting the image to Grey-scale
    newimage = cv.cvtColor(img,cv.cv2.COLOR_BGR2GRAY)
    
    #Resize the image to a square
    old_size = newimage.shape  # old_size[0] is in (width, height) format
    ratio = float(size) / max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    newimage = imutils.resize(newimage, width = new_size[1])

    #Create a new image and paste the resized on it
    totw = size - new_size[1]
    toth = size - new_size[0]
    delta_w = (size - new_size[1])//2
    delta_h = (size - new_size[0])//2
    padding = [delta_h, toth - delta_h, delta_w,totw - delta_w]
    
    #Creating the new image file and saving it.
    newimage = cv.copyMakeBorder(newimage,padding[0],padding[1],padding[2],padding[3],cv.BORDER_CONSTANT,value=0)
    greyname = 'grey' + filename
    cv.imwrite(greyname, newimage)

    #Read this new image for prediction.
    image_data = skimage.io.imread(greyname)
    new_image_data = skimage.transform.resize(image_data,(150,150,1))
    new_image_data = new_image_data.reshape((1, 150, 150, 1)).astype(np.float32) / 255.0
    (cn0, cn1, cn2) = model.predict(new_image_data)[0]

    #Outcome of the prediction is checked and the counters are modified accordingly.
    if cn0 > cn1 and cn0 > cn2 and class_names[0] in image: #and cn0 > cn3 and cn0 > cn4:
        tcn0+=1;
    elif cn1 > cn0 and cn1 > cn2 and class_names[1] in image: #and cn1 > cn3 and cn1 > cn4:
        tcn1+=1;
    elif cn2 > cn0 and cn2 > cn1 and class_names[2] in image: #and cn2 > cn3 and cn2 > cn4:
        tcn2+=1;
        
    
    #Printing the prediction value along with the file name
    print ('\nImage file name: ' + image)
    print (class_names[0] + ': ' + str(cn0))
    print (class_names[1] + ': ' + str(cn1))
    print (class_names[2] + ': ' + str(cn2))

    #Delete the created file.
    os.remove(greyname)
        
    return tcn0,acn0,tcn1,acn1,tcn2,acn2,tcn3,acn3,tcn4,acn4;

In [9]:
#Loading the model along with the weights file that was created by training
model = loadModel(modelFileName, weightFileName)

C:\ProgramData\Anaconda3\lib\site-packages\keras\models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Loaded model from disk


In [11]:
#Initializing the counters
acn0 = 0;
acn1 = 0;
acn2 = 0;
acn3 = 0;
acn4 = 0;
tcn0 = 0;
tcn1 = 0;
tcn2 = 0;
tcn3 = 0;
tcn4 = 0;

#Loop through all the files in the folder and predict.
for filename in os.listdir(testImageFolder):
    tcn0,acn0,tcn1,acn1,tcn2,acn2,tcn3,acn3,tcn4,acn4 = predictGrey(testImageFolder + '//' + filename,tcn0,acn0,tcn1,acn1,tcn2,acn2,tcn3,acn3,tcn4,acn4)
    
#Print the total correct predictions for each class.
print('\n'+class_names[0] +': ' + str(tcn0) + '/' + str(acn0));
print(class_names[1] +': ' + str(tcn1) + '/' + str(acn1));
print(class_names[2] +': ' + str(tcn2) + '/' + str(acn2));

C:\Users\Prem Thomas Eapen\AppData\Roaming\Python\Python36\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "



Image file name: Flickr27//TestImages//161270014-Apple.jpg
Apple: 0.021079872
Fedex: 0.9789201
Google: 2.4327347e-13

Image file name: Flickr27//TestImages//2276226904-Fedex.jpg
Apple: 4.3142344e-11
Fedex: 1.0
Google: 1.1230742e-15

Image file name: Flickr27//TestImages//2286135438-Fedex.jpg
Apple: 0.00016154717
Fedex: 0.9998374
Google: 1.0415865e-06

Image file name: Flickr27//TestImages//2550417833-Google.jpg
Apple: 0.00012949664
Fedex: 0.0006760823
Google: 0.99919444

Image file name: Flickr27//TestImages//2758671363-Google.jpg
Apple: 2.2320405e-10
Fedex: 9.617343e-09
Google: 1.0

Image file name: Flickr27//TestImages//2770852623-Google.jpg
Apple: 3.3422782e-09
Fedex: 9.68908e-08
Google: 0.9999999

Image file name: Flickr27//TestImages//2777979648-Google.jpg
Apple: 2.981311e-07
Fedex: 2.054687e-06
Google: 0.9999976

Image file name: Flickr27//TestImages//3276332721-Apple.jpg
Apple: 0.003114011
Fedex: 0.9964551
Google: 0.000430981

Image file name: Flickr27//TestImages//3579847658-G